In [ ]:
import csv
import os
from scenedetect import detect, ContentDetector, split_video_ffmpeg

# Set the base name
base_name = "Cigarettes"

# Construct file and directory names
video_file = f'{base_name}.mp4'
csv_filename = f'{base_name}_scene_detect.csv'
output_directory = f'{base_name}_split_videos'

os.makedirs(output_directory, exist_ok=True)

def get_scene_info(scene):
    start_frame, start_timecode = int(scene[0].get_frames()), scene[0].get_timecode()
    end_frame, end_timecode = int(scene[1].get_frames()), scene[1].get_timecode()

    # Calculate lengths
    length_frames = end_frame - start_frame
    length_timecode_frames = int(scene[1].get_frames()) - int(scene[0].get_frames())
    length_timecode_seconds = scene[1].get_seconds() - scene[0].get_seconds()
    length_seconds = scene[1].get_seconds() - scene[0].get_seconds()

    return start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, scene[0].get_seconds(), scene[1].get_seconds(), length_timecode_seconds, length_seconds

with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Video Name', 'Scene Number', 'Start Frame', 'End Frame', 'Length (frames)', 'Start Timecode', 'End Timecode', 'Length (timecode)', 'Start Time (seconds)', 'End Time (seconds)', 'Length (seconds)'])

    for i, scene in enumerate(detect(video_file, ContentDetector())):
        split_video_name = f'{base_name}-Scene-{i + 1:03d}.mp4'
        print(f'Splitting scene {i + 1}: {split_video_name}')
        split_video_ffmpeg(video_file, [scene], output_file_template=os.path.join(output_directory, split_video_name), show_progress=True)

        start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_timecode_seconds, length_seconds = get_scene_info(scene)
        csv_writer.writerow([split_video_name, i + 1, start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_seconds])


print(f'CSV file "{csv_filename}" has been created.')
print(f'Split videos are saved in the "{output_directory}" directory.')
